
## Model Training

In [ ]:
# Trained the model with model_main_tf2.py script present in Tensorflow Github repository
!python /projects/rkraig/phoenix_venv/TensorFlow/models/research/object_detection/model_main_tf2.py 
--pipeline_config_path=/projects/rkraig/phoenix_venv/config_files/Efficientdet.config 
--model_dir=/projects/rkraig/phoenix_venv/Efficientdet_Training_2018
--alsologtostderr 

## Model Testing

In [ ]:
#Import modules
import cv2, time, os, tensorflow as tf
import numpy as np
from tensorflow.python.keras.utils.data_utils import get_file
import shutil
from PIL import Image
import os
import glob


np.random.seed(26)


class Detector:
    def _init_(self):
        pass
    def readClasses(self, classesFilePath):
        with open(classesFilePath, 'r') as f:
            self.classesList = f.read().splitlines()
        
        #colors list
        self.colorList = np.random.uniform(low = 0, high = 255, size = (len(self.classesList), 3))
    
    # Option for Download model
    def downloadModel(self, modelURL):
        
        fileName = os.path.basename(modelURL)
        self.modelName = fileName[:fileName.index('.')]
    
        self.cacheDir = "/projects/rkraig/phoenix_venv/Object_detection/pretrained_models"
        
        os.makedirs(self.cacheDir)
        
        get_file(fname = fileName, 
                origin = modelURL, cache_dir = self.cacheDir, cache_subdir = "checkpoints", extract = True)
    
    
    # Load the downloaded model or saved model
    def loadModel(self):
        print("Loading Model " + self.modelName)
        tf.keras.backend.clear_session()
        self.model = tf.saved_model.load(os.path.join(self.cacheDir, "checkpoints", self.modelName, "saved_model"))
        
        print("Model " + self.modelName + "loaded successfully...")
        
    def createBoundingBox(self, image, threshold = 0.5):
        inputTensor = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
        inputTesnor = tf.convert_to_tensor(inputTensor, dtype = tf.uint8)
        inputTensor = inputTensor[tf.newaxis, ...]
        
        detections = self.model(inputTensor)
        
        bboxs = detections['detection_boxes'][0].numpy() 
        classIndexes = detections['detection_classes'][0].numpy().astype(np.int32)
        classScores = detections['detection_scores'][0].numpy()
        
        
        imH, imW, imC = image.shape
        
        bboxIdx = tf.image.non_max_suppression(bboxs, classScores, max_output_size= 50,
                                              iou_threshold = threshold, score_threshold = threshold)
        print(bboxIdx)
        
        if len(bboxIdx) != 0:
            for i in bboxIdx:
                bbox = tuple(bboxs[i].tolist())
                classConfidence = round(100*classScores[i])
                classIndex = classIndexes[i]
                
                classLabelText = self.classesList[classIndex].upper()
                classColor = self.colorList[classIndex]
                
                displayText = '{}: {}%'.format(classLabelText, classConfidence)
                
                ymin, xmin, ymax, xmax = bbox
                
                xmin, xmax, ymin, ymax = (xmin * imW, xmax * imW, ymin * imH, ymax * imH)
                xmin, xmax, ymin, ymax = int(xmin), int(xmax), int(ymin), int(ymax)
                
                cv2.rectangle(image, (xmin, ymin), (xmax,ymax), color = classColor, thickness = 1)
                cv2.putText(image, displayText, (xmin, ymin - 10), cv2.FONT_HERSHEY_PLAIN, 1, classColor, 2)
                
                ###############################################
                
                lineWidth = min(int((xmax-xmin)*0.2), int ((ymax - ymin)*0.2))
                
                cv2.line(image, (xmin, ymin), (xmin + lineWidth, ymin), classColor, thickness = 5)
                cv2.line(image, (xmin, ymin), (xmin, ymin + lineWidth), classColor, thickness = 5)
                
                cv2.line(image, (xmax, ymin), (xmax - lineWidth, ymin), classColor, thickness = 5)
                cv2.line(image, (xmax, ymin), (xmax, ymin + lineWidth), classColor, thickness = 5)
        
                ###############################################
                
                cv2.line(image, (xmin, ymax), (xmin + lineWidth, ymax), classColor, thickness = 5)
                cv2.line(image, (xmin, ymax), (xmin, ymax - lineWidth), classColor, thickness = 5)
                
                cv2.line(image, (xmax, ymax), (xmax - lineWidth, ymax), classColor, thickness = 5)
                cv2.line(image, (xmax, ymax), (xmax, ymax - lineWidth), classColor, thickness = 5)
                
            return image,classConfidence,classLabelText   
        
        else:
            return image, None, None
    
        
    def predictImage(self, imagesPath, threshold = 0.5):
        ImgItems = os.listdir(imagesPath)
        # Create a dataframe to store filename, Hydrant_presence(yes/no) and confidence scores
        df = pd.DataFrame(columns = ['Name', 'Hydrant_presence','Conf_Scores'])
        for item in ImgItems:
        # Open the image
             img_path = os.path.join(imagesPath,item)
             img = cv2.imread(img_path)
             bboxImage,conf,lb = self.createBoundingBox(img, threshold)
             if conf!= None:
                    df = df.append({'Name' :item, 'Hydrant' :'YES', 'Conf_Scores' :conf},ignore_index=True)
             else:
                    df = df.append({'Name' :item, 'Hydrant' :'NO', 'Conf_Scores' :0},ignore_index=True)
             #cv2.imwrite(self.modelName + ".jpg", bboxImage)
             path="/projects/rkraig/phoenix_venv/Object_detection/Output_coco_hydrant"
    
             cv2.imwrite(os.path.join(path ,item), bboxImage)
             #cv2.imwrite(os.path.join(path ,self.modelName + "_"+f"{count}_.jpg"), bboxImage)
        
         return df

In [ ]:
#modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz"

classFile = "/projects/rkraig/phoenix_venv/Object_detection/coco_hydrant.txt"
imagesPath = "/projects/rkraig/phoenix_venv/Object_detection/Evaluation/test_images"
threshold = 0.5

detector = Detector()
detector.readClasses(classFile)
detector.downloadModel(modelURL)
detector.loadModel()
df=detector.predictImage(imagesPath, threshold)

## Model Evaluation using Confusion matrix

In [ ]:
# Get the ground truth values
gr_truth=pd.read_csv('/home/vadabala/Desktop/projects/rkraig/phoenix_venv/hydrant_169.csv',sep=',')

In [ ]:
# Get the info about hydrant presence images
df1=df[(df.Hydrant=='YES')]

In [ ]:
len(df1)

In [ ]:
# Average confidence score of detected hydrants
df1.Conf_Scores.mean()

#### Confusion matrix values

In [ ]:
#To determine the false positives
fp=0
fp_list=[]
for f in list(df1.Name):
    if f not in list(gr_truth.filename):
        fp=fp+1
        fp_list.append(f)
print(fp)

In [ ]:
#To determine true positives
tp=len(df1)
not_tp_list=[]
for f in list(df1.Name):
    if f not in list(gr_truth.filename):
        tp=tp-1
        not_tp_list.append(f)
print(tp)

In [ ]:
#To determine false negatives
fn=0
fn_list=[]
for f in list(gr_truth.filename):
    #print(f)
    if f not in list(df1.Name):
        fn+=1
        fn_list.append(f)
print(fn)    

In [ ]:
#To determine true negatives
tn=len(df2)
not_tn_list=[]
for f in list(df2.Name):
    if f in list(gr_truth.filename):
        tn=tn-1
        not_tn_list.append(f)
print(tn)

In [ ]:
# Get the precision
precision= tp/(tp+fp)

In [ ]:
# Get the recall
recall= tp/(tp+fn)